<a href="https://colab.research.google.com/github/hybrits/H-ALLY/blob/main/Primer_entramientoV_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import spacy
from nltk.corpus import stopwords
import nltk

# Descargar stopwords si no están ya descargadas
nltk.download('stopwords')

# Cargar el archivo CSV
file_path = '/content/foro_data_paginas_1_a_10 (1).csv'
df = pd.read_csv(file_path)

# Eliminar columnas no deseadas
columns_to_drop = ['URL', 'Autor', 'Fecha']
df = df.drop(columns=columns_to_drop, errors='ignore')

# Cargar el modelo de spaCy para inglés
nlp = spacy.load('en_core_web_sm')

# Lista de palabras vacías (stop words) en inglés
stop_words = set(stopwords.words('english'))

def limpiar_texto_ingles(texto):
    # Procesar el texto con spaCy
    doc = nlp(texto)
    # Filtrar tokens, eliminando palabras vacías y signos de puntuación
    tokens_limpios = [token.lemma_ for token in doc if token.text.lower() not in stop_words and not token.is_punct]
    # Unir tokens limpios en un solo string
    texto_limpio = ' '.join(tokens_limpios)
    return texto_limpio

# Aplicar la función de limpieza a las columnas "Título" y "Contenido" usando stopwords en inglés
df['Título'] = df['Título'].astype(str).apply(limpiar_texto_ingles)
df['Contenido'] = df['Contenido'].astype(str).apply(limpiar_texto_ingles)

# Guardar el DataFrame limpio en un nuevo archivo CSV
output_file_path_english = '/content/foro_data_paginas_limpio_ingles.csv'
df.to_csv(output_file_path_english, index=False)

output_file_path_english


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'/content/foro_data_paginas_limpio_ingles.csv'

In [4]:
import pandas as pd

# Intentar cargar el dataset con diferentes codificaciones y parámetros
codificaciones = ['utf-8', 'latin-1', 'iso-8859-1']

for codificacion in codificaciones:
    try:
        df = pd.read_csv('/content/foro_data_paginas_limpio_ingles.csv', encoding=codificacion, delimiter=',', quotechar='"', on_bad_lines='skip')
        print(f"El archivo se cargó correctamente con la codificación {codificacion}")
        break
    except UnicodeDecodeError:
        print(f"Error al cargar el archivo con la codificación {codificacion}")
    except pd.errors.ParserError:
        print(f"Error al analizar el archivo con la codificación {codificacion}")

# Ver las primeras filas del dataset
print(df.head())

# Ver los nombres de las columnas
print(df.columns)

# Eliminar filas con valores nulos
df = df.dropna()

# Eliminar duplicados
df = df.drop_duplicates()

# Verificar la limpieza
print(df.info())

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Función para preprocesar texto
def preprocess_text(text):
    # Tokenización
    tokens = word_tokenize(text.lower())
    # Eliminación de stop words
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Aplicar preprocesamiento a la columna de texto
df['Contenido'] = df['Contenido'].apply(preprocess_text)

!pip install transformers

from transformers import BertTokenizer

# Cargar el tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenización de los datos
train_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)

import torch

# Crear un dataset compatible con PyTorch
class ProblemDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convertir las etiquetas a enteros (si es necesario)
label_mapping = {label: idx for idx, label in enumerate(df['Título'].unique())}
df['Título'] = df['Título'].map(label_mapping)

# Dividir el dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Contenido'], df['Título'], test_size=0.2, random_state=42)

train_dataset = ProblemDataset(train_encodings, y_train)
test_dataset = ProblemDataset(test_encodings, y_test)

from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Cargar el modelo
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping))

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Entrenamiento del modelo
trainer.train()

# Evaluación del modelo
results = trainer.evaluate()
print(results)


El archivo se cargó correctamente con la codificación utf-8
                          Título  \
0               overclock i2500k   
1               overclock i2500k   
2      overclock core 2 duo quad   
3      overclock core 2 duo quad   
4  new overclocking read starter   

                                           Contenido  
0  say post go overclocking thread overclock core...  
1                               contenido encontrado  
2  first let warn overclocke voids warranty creat...  
3                               contenido encontrado  
4  want overclock \n\n\n\n assist properly clock ...  
Index(['Título', 'Contenido'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 3344 entries, 0 to 3361
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Título     3344 non-null   object
 1   Contenido  3344 non-null   object
dtypes: object(2)
memory usage: 78.4+ KB
None


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 110

In [5]:
df = pd.read_csv('/content/foro_data_paginas_limpio_ingles.csv')

In [6]:
print(df.columns)
print(df.head())


Index(['Título', 'Contenido'], dtype='object')
                          Título  \
0               overclock i2500k   
1               overclock i2500k   
2      overclock core 2 duo quad   
3      overclock core 2 duo quad   
4  new overclocking read starter   

                                           Contenido  
0  say post go overclocking thread overclock core...  
1                               contenido encontrado  
2  first let warn overclocke voids warranty creat...  
3                               contenido encontrado  
4  want overclock \n\n\n\n assist properly clock ...  


In [7]:
if 'Título' in df.columns:
    print("La columna 'Título' está presente.")
else:
    print("La columna 'Título' no está presente.")

La columna 'Título' está presente.


In [8]:
print(df['Título'].unique())


['overclock i2500k' 'overclock core 2 duo quad'
 'new overclocking read starter' 'RX 7700 XT GPU 5 7600 CPU bottleneck'
 'good question' 'question undervolt' 'pc heavily Underperforming Upgrade'
 'enable XMP cause secureboot failure'
 'Corsair Vengeance DDR4 3400 @2133 mhz' 'upgrade cpu r5 2600 R5 5600X'
 'new Overclocking 13700k' 'black screen fail OC'
 'computer restart self play game' 'ryzen 5 5600X keep i5 8600 k'
 'new graphic card give nothing problem'
 'balance performance cpu temperature' 'win 10 Restore Error'
 'look software like SpeedFan' 'system monitor' 'safe overclock RTX 3050'
 'AMD FX-4100' 'overclocke issue RTX 3070 laptop gpu help'
 'pc will not boot XMP profile'
 'B450 gaming PLUS MAX cpu overclocke issue' 'lag fps stutter plug'
 'underclocking setting use longevity rtx 2080'
 'try overclock however can not select ratio grey'
 'help overclock i7 4510U' 'huge cpu performance drop CPU overclock'
 'GPU memory problem' 'amd ryzen break pc' 'question undervolte CPU'
 '8 g

In [10]:
df = df.reset_index(drop=True)


In [11]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))


2675 2675
669 669


In [12]:
train_dataset = ProblemDataset(train_encodings, y_train)
test_dataset = ProblemDataset(test_encodings, y_test)


In [13]:
# Inspeccionar una muestra de los datos y etiquetas
print(df.iloc[110])  # Revisar la fila que parece causar el problema


Título                             look software like SpeedFan
Contenido    7 Best Speedfan Alternatives 2023 | Technize \...
Name: 110, dtype: object


In [14]:
# Verificar el mapeo de etiquetas
label_mapping = {label: idx for idx, label in enumerate(df['Título'].unique())}
print(label_mapping)

# Aplicar el mapeo
df['Título'] = df['Título'].map(label_mapping)


{'overclock i2500k': 0, 'overclock core 2 duo quad': 1, 'new overclocking read starter': 2, 'RX 7700 XT GPU 5 7600 CPU bottleneck': 3, 'good question': 4, 'question undervolt': 5, 'pc heavily Underperforming Upgrade': 6, 'enable XMP cause secureboot failure': 7, 'Corsair Vengeance DDR4 3400 @2133 mhz': 8, 'upgrade cpu r5 2600 R5 5600X': 9, 'new Overclocking 13700k': 10, 'black screen fail OC': 11, 'computer restart self play game': 12, 'ryzen 5 5600X keep i5 8600 k': 13, 'new graphic card give nothing problem': 14, 'balance performance cpu temperature': 15, 'win 10 Restore Error': 16, 'look software like SpeedFan': 17, 'system monitor': 18, 'safe overclock RTX 3050': 19, 'AMD FX-4100': 20, 'overclocke issue RTX 3070 laptop gpu help': 21, 'pc will not boot XMP profile': 22, 'B450 gaming PLUS MAX cpu overclocke issue': 23, 'lag fps stutter plug': 24, 'underclocking setting use longevity rtx 2080': 25, 'try overclock however can not select ratio grey': 26, 'help overclock i7 4510U': 27, '

In [17]:
print(df.isnull().sum())


Título       0
Contenido    7
dtype: int64


In [19]:
df['Contenido'] = df['Contenido'].fillna('')


In [18]:
df = df.dropna(subset=['Contenido'])

In [15]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))


2675 2675
669 669


In [20]:
nan_rows = df[df['Contenido'].isna()]
print(nan_rows)


Empty DataFrame
Columns: [Título, Contenido]
Index: []


In [21]:
# Ejemplo de preparación de datos
train_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)
df['Título'] = df['Título'].map(label_mapping).astype(int)  # Asegúrate de que las etiquetas sean enteros

X_train, X_test, y_train, y_test = train_test_split(train_encodings['input_ids'], df['Título'], test_size=0.2, random_state=42)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [22]:
nan_titles = df['Título'].map(label_mapping).isna()
print(df[nan_titles])


      Título                                          Contenido
0          0  say post go overclocking thread overclock core...
1          0                               contenido encontrado
2          1  first let warn overclocke voids warranty creat...
3          1                               contenido encontrado
4          2  want overclock \n\n\n\n assist properly clock ...
...      ...                                                ...
3357     345  section overclocking feature concern due high ...
3358     345                               contenido encontrado
3359     345  Epox 9nda3i Power BIOS \n\n Default Settings o...
3360     345  Epox 9nda3i Power BIOS \n\n hope help \n\n mob...
3361     345  Epox 9nda3i Power BIOS \n\n check Temps Voltag...

[3355 rows x 2 columns]


In [23]:
df['Título'] = df['Título'].map(label_mapping)
df['Título'].fillna(-1, inplace=True)  # Asignar una etiqueta predeterminada, por ejemplo -1
df['Título'] = df['Título'].astype(int)


In [26]:
print(df['Título'].isna().sum())

0


In [25]:
# Ejemplo de preparación de datos
train_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)
df['Título'] = df['Título'].map(label_mapping).astype(int)  # Asegúrate de que las etiquetas sean enteros

X_train, X_test, y_train, y_test = train_test_split(train_encodings['input_ids'], df['Título'], test_size=0.2, random_state=42)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

# Cargar el archivo CSV
df = pd.read_csv('/content/foro_data_paginas_limpio_ingles.csv')

# Crear un mapeo de etiquetas único para cada valor en la columna 'Título'
label_mapping = {label: idx for idx, label in enumerate(df['Título'].unique())}

# Aplicar el mapeo de etiquetas a la columna 'Título'
df['Título'] = df['Título'].map(label_mapping)

# Verificar los valores NaN en 'Título'
nan_titles = df['Título'].isna()
print("Filas con NaN en 'Título':")
print(df[nan_titles])

# Manejar valores NaN: Opción 1 - Eliminar filas con NaN
df = df.dropna(subset=['Título'])

# Convertir 'Título' a enteros después de manejar NaN
df['Título'] = df['Título'].astype(int)

# Verificar que no queden NaN en 'Título'
print("Número de valores NaN en 'Título' después de la limpieza:", df['Título'].isna().sum())

# Tokenización usando DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(train_encodings['input_ids'], df['Título'], test_size=0.2, random_state=42)

print("Datos preparados para el entrenamiento.")


Filas con NaN en 'Título':
Empty DataFrame
Columns: [Título, Contenido]
Index: []
Número de valores NaN en 'Título' después de la limpieza: 0


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

# Cargar el archivo CSV
df = pd.read_csv('/content/foro_data_paginas_limpio_ingles.csv')

# Crear un mapeo de etiquetas único para cada valor en la columna 'Título'
label_mapping = {label: idx for idx, label in enumerate(df['Título'].unique())}

# Aplicar el mapeo de etiquetas a la columna 'Título'
df['Título'] = df['Título'].map(label_mapping)

# Verificar los valores NaN en 'Título'
nan_titles = df['Título'].isna()
print("Filas con NaN en 'Título':")
print(df[nan_titles])

# Manejar valores NaN: Opción 1 - Eliminar filas con NaN
df = df.dropna(subset=['Título'])

# Convertir 'Título' a enteros después de manejar NaN
df['Título'] = df['Título'].astype(int)

# Verificar que no queden NaN en 'Título'
print("Número de valores NaN en 'Título' después de la limpieza:", df['Título'].isna().sum())

# Asegurarse de que todos los valores en 'Contenido' sean cadenas de texto
df['Contenido'] = df['Contenido'].fillna('').astype(str)

# Tokenización usando DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(train_encodings['input_ids'], df['Título'], test_size=0.2, random_state=42)

print("Datos preparados para el entrenamiento.")


Filas con NaN en 'Título':
Empty DataFrame
Columns: [Título, Contenido]
Index: []
Número de valores NaN en 'Título' después de la limpieza: 0
Datos preparados para el entrenamiento.


In [32]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Verificar si hay una GPU disponible y configurarla para el entrenamiento
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Crear el modelo
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_mapping))
model.to(device)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida para guardar los resultados
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=8,   # Tamaño del lote por dispositivo durante el entrenamiento
    per_device_eval_batch_size=8,    # Tamaño del lote por dispositivo durante la evaluación
    warmup_steps=500,                # Número de pasos para el calentamiento
    weight_decay=0.01,               # Tasa de decaimiento del peso
    logging_dir='./logs',            # Directorio de registro para TensorBoard
    logging_steps=10,
    evaluation_strategy="epoch"      # Evaluación al final de cada época
)

# Convertir las entradas y etiquetas de entrenamiento y prueba a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Crear un dataset a partir de las entradas y etiquetas
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Crear datasets de entrenamiento y prueba
train_dataset = CustomDataset({'input_ids': X_train, 'attention_mask': train_encodings['attention_mask'][:len(X_train)]}, y_train)
test_dataset = CustomDataset({'input_ids': X_test, 'attention_mask': train_encodings['attention_mask'][len(X_train):]}, y_test)

# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
trainer.evaluate()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: could not determine the shape of object type 'Series'

In [ ]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Verificar si hay una GPU disponible y configurarla para el entrenamiento
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Crear el modelo
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_mapping))
model.to(device)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida para guardar los resultados
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=8,   # Tamaño del lote por dispositivo durante el entrenamiento
    per_device_eval_batch_size=8,    # Tamaño del lote por dispositivo durante la evaluación
    warmup_steps=500,                # Número de pasos para el calentamiento
    weight_decay=0.01,               # Tasa de decaimiento del peso
    logging_dir='./logs',            # Directorio de registro para TensorBoard
    logging_steps=10,
    eval_strategy="epoch"            # Evaluación al final de cada época
)

# Convertir las entradas y etiquetas de entrenamiento y prueba a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.long)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Crear un dataset a partir de las entradas y etiquetas
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Crear datasets de entrenamiento y prueba
train_dataset = CustomDataset({'input_ids': X_train, 'attention_mask': train_encodings['attention_mask'][:len(X_train)]}, y_train)
test_dataset = CustomDataset({'input_ids': X_test, 'attention_mask': train_encodings['attention_mask'][len(X_train):]}, y_test)

# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
trainer.evaluate()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-33-6c85cb76e38f>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.long)
<ipython-input-33-6c85cb76e38f>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.long)
<ipython-input-33-6c85cb76e38f>:37: UserWarning: To copy construct from a tensor,

Epoch,Training Loss,Validation Loss
